In [1]:
!pip install -Uq langchain faiss-cpu langchain-community langchain-core langchain-groq PyMuPDF tiktoken bs4 fastembed tavily-python rerankers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 9

In [2]:
import os
import time
import warnings
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from tavily.errors import InvalidAPIKeyError
from langchain.schema.runnable import Runnable
from typing import cast

In [3]:
warnings.filterwarnings('ignore')

In [4]:
def search_web(query: str) -> list:
    """Search the web."""
    try:
        tavily_tool = TavilySearchAPIRetriever(
            k=10,
            #api_key=os.getenv("TAVILY_API_KEY"),
            api_key =userdata.get('TAVILY_API_KEY'),
            max_tokens=10000,
            search_depth='advanced'
        )
        results = tavily_tool.invoke(query)
        if not results:
            return ["No relevant context found."]
        return [result.page_content for result in results]
    except InvalidAPIKeyError as e:
        return ["Invalid API key."]
    except Exception as e:
        return [f"Error occurred while fetching context: {str(e)}"]

In [5]:
# groq_key = os.getenv("GROQ_API_KEY")
# tavily_key = os.getenv("TAVILY_API_KEY")
from google.colab import userdata
groq_key = userdata.get('GROQ_API_KEY')


In [6]:
llm = ChatGroq(
        api_key=groq_key,
        model='llama-3.2-1b-preview',
        temperature=0,
        max_retries=3,
        timeout=None,
        max_tokens=1024
    )
llm.invoke("What is AI")

AIMessage(content='Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:\n\n1. Learning: AI systems can learn from data and improve their performance over time.\n2. Problem-solving: AI systems can analyze data, identify patterns, and make decisions.\n3. Reasoning: AI systems can draw conclusions based on data and rules.\n4. Perception: AI systems can interpret and understand data from sensors and other sources.\n5. Natural Language Processing (NLP): AI systems can understand and generate human language.\n\nAI can be categorized into several types, including:\n\n1. Narrow or Weak AI: Designed to perform a specific task, such as facial recognition or language translation.\n2. General or Strong AI: Designed to perform any intellectual task that a human can, such as reasoning, problem-solving, and learning.\n3. Superintelligence: Significantly more intelligent than the best human minds, with cap

In [7]:
prompt = """You are a helpful assistant.

            Answer the question according to the query and given context:
            Question: {question}
            Context: {context}
            Provide an accurate response in bullet points but don't mention it in the response,
            the answer should be brief (max 5 lines/points).
            Do not hallucinate.
    """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [8]:
chain = prompt_template | llm

In [9]:
query = input("")
context = search_web(query)
response = chain.invoke({'question':query,
                         'context':context})
response.content

what is ai


"Here's an accurate response in bullet points:\n\n• AI, also known as Artificial intelligence, is a technology with human-like problem-solving capabilities.\n• AI is a set of technologies that enable computers to perform a variety of advanced functions.\n• Machine learning, generative AI, and deep learning are key areas of AI research.\n• AI has various applications, including image recognition, natural language processing, and predictive analytics.\n• The field of AI is constantly evolving, with new breakthroughs and challenges emerging regularly."

## Method 1

In [11]:
!pip install rank_bm25

In [14]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split() for doc in context]
bm25 = BM25Okapi(tokenized_corpus)
scores = bm25.get_scores(query.split())
scored_docs = sorted(zip(context, scores), key=lambda x: x[1], reverse=True)
top3_doc=[doc for doc, _ in scored_docs[:3]]
response = chain.invoke({'question':query,
                         'context':top3_doc})
response.content

"Here's an accurate response in bullet points:\n\n• Artificial intelligence (AI) refers to a set of technologies that enable computers to perform advanced functions, including language understanding, data analysis, and decision-making.\n• AI is a field of science concerned with building computers and machines that can reason, learn, and act in a way that would normally require human intelligence.\n• Machine learning and deep learning are two primary types of AI, used for data analytics, predictions, and forecasting, object categorization, and natural language processing.\n• AI can be used in various applications, including chatbots, computer vision, and natural language processing, to perform tasks that previously only humans can do.\n• The development and deployment of AI technologies are driven by the increasing availability of data, computing power, and machine learning algorithms."

## Method 2

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(context)
query_vector = vectorizer.transform([query])
similarity = cosine_similarity(query_vector,doc_vectors).flatten()
scored_docs = sorted(zip(context, similarity), key=lambda x: x[1], reverse=True)
top3_doc = [doc for doc, _ in scored_docs[:3]]

['Solution\nSolution\nSolution\nSolution\nSolution\nSolution\nTake the next step\nStart\nbuilding on Google Cloud with $300 in free credits and 20+\nalways free products.\n Artificial intelligence (AI) is a set of technologies\nthat enable computers to perform a variety of advanced\nfunctions, including the ability to\nsee,\nunderstand and\ntranslate spoken and written language,\nanalyze data,\nmake recommendations, and more.\n Artificial intelligence\ndefined\nArtificial intelligence is a field of science\nconcerned with building computers and machines that\ncan reason, learn, and act in such a way that would\nnormally require human intelligence or that involves\ndata whose scale exceeds what humans can\nanalyze.\n By using products like\nVertex AI,\nCCAI,\nDocAI,\nor AI APIs, organizations can make sense of all the data\nthey’re producing, collecting, or otherwise analyzing,\nno matter what format it’s in, to make actionable\nbusiness decisions.\n On an operational level for business

## Method 3

In [39]:
!pip install transformers

In [48]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer(
        [(query, doc) for doc in context],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}
with torch.no_grad():
      scores = model(**inputs).logits.squeeze(-1)
scored_docs = sorted(zip(context, scores.tolist()), key=lambda x: x[1], reverse=True)
top3_doc = [doc for doc, _ in scored_docs[:3]]

['AI, also known as Artificial intelligence, is a technology with human-like problem-solving capabilities. AI in action appears to simulate human intelligence—it can recognize images, write poems, and make data-based predictions.',
 'AI is a technical and scientific field that generates outputs for human-defined objectives using data analysis, machine learning and reasoning. Learn about the types, applications and impact of AI, as well as the history and governance of the technology.',
 'Artificial Intelligence (AI) is the development of computer systems that perform tasks requiring human intelligence. Learn about the evolution, types, applications, and future of AI, as well as its importance and challenges.']

## Method 4

In [49]:
!pip install sentence-transformers

In [53]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
query_embedding = model.encode(query, convert_to_tensor=True)
doc_embeddings = model.encode(context, convert_to_tensor=True)
similarities = util.cos_sim(query_embedding, doc_embeddings)[0].cpu().numpy()
scored_docs = sorted(
        zip(context, similarities), key=lambda x: x[1], reverse=True
    )
top3_doc = [doc for doc, _ in scored_docs[:3]]

['AI, also known as Artificial intelligence, is a technology with human-like problem-solving capabilities. AI in action appears to simulate human intelligence—it can recognize images, write poems, and make data-based predictions.',
 'Artificial Intelligence (AI) is the development of computer systems that perform tasks requiring human intelligence. Learn about the evolution, types, applications, and future of AI, as well as its importance and challenges.',
 'AI is a technical and scientific field that generates outputs for human-defined objectives using data analysis, machine learning and reasoning. Learn about the types, applications and impact of AI, as well as the history and governance of the technology.']

## Method 5

In [54]:
!pip install spacy

In [66]:
import spacy

nlp = spacy.load("en_core_web_sm")
query_doc = nlp(query)
scored_docs = sorted(
        context,
        key=lambda doc: query_doc.similarity(nlp(doc)),
        reverse=True,
    )